# Neo4j와 LangChain을 활용한 영화 추천 시스템

---

## 1. Neo4J DB 환경 설정

In [1]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv(override=True)

True

In [2]:
from langchain_neo4j import Neo4jGraph

# LangChain 도구 활용 - DB 연결 객체 초기화 
graph = Neo4jGraph( 
    url=os.getenv("NEO4J_URI"), 
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD"),
    database=os.getenv("NEO4J_DATABASE"),
    enhanced_schema=True,   # 향상된 스키마 사용 설정
)

In [3]:
# 테스트 쿼리 실행 
cypher_query = """
MATCH (n:Movie)
RETURN COUNT(n) AS Movie_Count
"""

graph.query(cypher_query)

[{'Movie_Count': 2569}]

---

## 2. Neo4j 기반 질의 응답 시스템 (GraphRAG) 구현하기

Graph 데이터베이스를 기반으로 한 질의 응답 시스템(GraphRAG)은 전통적인 벡터 기반 RAG 시스템보다 더 정확하고 연관성 있는 답변을 제공할 수 있습니다. 이 방식의 핵심은 자연어 질문을 Neo4j Cypher 쿼리로 변환하여 지식 그래프를 효과적으로 탐색하는 것입니다. 

- **특장점**:

   - **정확한 관계 검색**: 그래프 데이터베이스의 관계 중심 구조를 활용해 복잡한 연결 패턴을 찾을 수 있습니다
   - **컨텍스트 유지**: 엔티티 간의 관계를 유지하여 더 풍부한 컨텍스트를 제공합니다
   - **구조화된 정보 검색**: 단순 텍스트 검색이 아닌 구조화된 방식으로 정보를 검색합니다

- **필요 사항**:
   - Neo4j 데이터베이스 (영화 데이터 포함)
   - 필요 패키지: `langchain-neo4j`, `langchain-openai`

### 2.1 벡터 검색(Semantic Search)


#### 1) **Graph DB 초기화**

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain_neo4j import Neo4jVector

# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small") 

# Neo4j 데이터베이스에 이미 생성된 벡터 인덱스에 연결하는 Neo4jVector 인스턴스 생성
graph_vector = Neo4jVector.from_existing_index(
    embeddings,  # 사용할 임베딩 모델 지정
    url=os.getenv("NEO4J_URI"),  # Neo4j 데이터베이스 연결 URI (환경 변수에서 가져옴)
    username=os.getenv("NEO4J_USERNAME"),  # Neo4j 데이터베이스 사용자 이름
    password=os.getenv("NEO4J_PASSWORD"),  # Neo4j 데이터베이스 비밀번호
    index_name="movie_content_embeddings",  # 사용할 벡터 인덱스 이름 (이미 Neo4j에 생성되어 있어야 함)
    text_node_property="overview",  # 텍스트 검색 시 반환할 노드의 속성 (영화 개요)
)

#### 2) **벡터 검색 (유사도 기준)**

In [5]:
# 한국어로 된 자연어 쿼리를 사용하여 의미적으로 유사한 영화 검색
query = "2차 세계대전을 배경으로 군인들의 활약상을 그린 영화를 찾아주세요."

# 유사도 검색 수행
similar_docs = graph_vector.similarity_search_with_score(
    query,
    k=5, # 유사도 상위 5개 문서 검색
    return_embeddings=False, # 임베딩 반환 안함 (결과 간소화)
)

# 각 문서와 해당 유사도 점수를 함께 표시
for doc, score in similar_docs:
    print(f"줄거리: {doc.page_content[:100]}..., 유사도: {score}")
    print("영화 제목:", doc.metadata.get("title"))
    print("-" * 50)  

줄거리: A US Fighter pilot's epic struggle of survival after being shot down on a mission over Laos during t..., 유사도: 0.7092428207397461
영화 제목: Rescue Dawn
--------------------------------------------------
줄거리: Wounded in Africa during World War II, Nazi Col. Claus von Stauffenberg returns to his native German..., 유사도: 0.7024703025817871
영화 제목: Valkyrie
--------------------------------------------------
줄거리: In a place where killers are celebrated as heroes, these filmmakers challenge unrepentant death-squa..., 유사도: 0.7021176815032959
영화 제목: The Act of Killing
--------------------------------------------------
줄거리: The Earth is attacked by unknown forces. As people everywhere watch the world's great cities fall, L..., 유사도: 0.7009596824645996
영화 제목: Battle: Los Angeles
--------------------------------------------------
줄거리: In Nazi-occupied France during World War II, a group of Jewish-American soldiers known as "The Baste..., 유사도: 0.7009172439575195
영화 제목: Inglourious Basterds
----------

#### 3) **그래프 경로를 활용한 검색 확장**

- 필모그래피(filmography) : 한 사람이 참여한 영화(또는 영상 작품)의 목록

In [6]:
# Neo4j 그래프 객체 생성
from langchain_neo4j import Neo4jGraph
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI"),
    username=os.getenv("NEO4J_USERNAME"),
    password=os.getenv("NEO4J_PASSWORD"),
    enhanced_schema=True,
)

In [7]:
def find_movies_and_actors_filmography(query, graph_db, k=5):
    """
    특정 영화에 출연한 배우들의 필모그래피를 찾는 함수
    
    Args:
        query: 검색 쿼리 (예: "2차 세계대전 영화")
        graph_db: 벡터 저장소 객체
        k: 검색할 영화 수
    
    Returns:
        영화와 배우 정보가 계층적으로 정리된 결과
    """
    # 1단계: 벡터 검색으로 영화 검색
    results = graph_vector.similarity_search(query, k=k)
    print(f"벡터스토어 검색된 영화 수: {len(results)}")
    
    # 검색된 영화 제목 추출
    movie_titles = []
    for doc in results:
        title = doc.metadata.get("title")
        if title:
            movie_titles.append(title)
    
    # 제목이 없으면 빈 결과 반환
    if not movie_titles:
        return {"movies": [], "message": "검색된 영화가 없습니다."}
   
  
    # 2단계: 영화와 출연 배우 정보 가져오기
    # Cypher 쿼리 설명:
    # 1. MATCH (movie:Movie): Movie 레이블을 가진 노드 찾기
    # 2. WHERE ANY(...): 검색된 영화 제목 중 하나라도 포함하는 영화 필터링
    # 3. OPTIONAL MATCH: 영화에 출연한 배우가 없을 수도 있으므로 OPTIONAL 사용
    # 4. collect(actor): 각 영화별로 출연 배우들을 배열로 수집
    # 5. RETURN: 영화 정보와 배우 정보를 함께 반환
    movies_actors_query = """
    // 영화 제목 중 하나라도 포함하는 영화 필터링
    MATCH (movie:Movie)
    WHERE ANY(title IN $movie_titles WHERE movie.title CONTAINS title)

    // 영화에 출연한 배우가 없을 수도 있으므로 OPTIONAL 사용
    OPTIONAL MATCH (movie)<-[acted:ACTED_IN]-(actor:Person)

    // 각 영화별로 출연 배우들을 배열로 수집
    WITH movie, collect(actor) as actors

    // 영화 정보와 배우 정보를 함께 반환
    RETURN 
        movie.title as movie_title,
        movie.released as release_date,
        movie.rating as rating,
        movie.overview as overview,
        [actor IN actors | actor.name] as actor_names,   // 배우 이름 배열
        [actor IN actors | {name: actor.name, id: id(actor)}] as actors  // 배우 정보 배열
    """
    
    # 영화와 출연 배우 정보 쿼리 실행
    # 벡터 db에서 검색한 movie_titles를 파라미터로 전달
    search_movies = graph.query(movies_actors_query, params={"movie_titles": movie_titles})
    
    # 배우 ID 목록 추출
    actor_ids = []
    for movie in search_movies:
        for actor in movie.get("actors", []):
            if "id" in actor:
                actor_ids.append(actor["id"])
    
    # 중복 제거 (동일한 배우가 여러 영화에 출연할 수 있음)
    actor_ids = list(set(actor_ids))
    
    # 배우가 없으면 영화 정보만 반환
    if not actor_ids:
        return {
            "movies": search_movies,
            "message": "검색된 영화에서 배우 정보를 찾을 수 없습니다."
        }
    
    # 3단계: 각 배우의 필모그래피 가져오기
    # Cypher 쿼리 설명:
    # 1. MATCH (actor:Person): 특정 ID를 가진 배우 노드 찾기
    # 2. WHERE id(actor) IN $actor_ids: 앞서 찾은 배우 ID 목록으로 필터링
    # 3. MATCH (actor)-[:ACTED_IN]->(movie:Movie): 배우가 출연한 영화 찾기
    # 4. WHERE NOT movie.title IN $movie_titles: 원래 검색된 영화는 제외 (중복 방지)
    # 5. WITH actor.name, collect(...): 배우별로 출연 영화 정보 수집
    # 6. RETURN: 배우 이름과 출연 영화 목록 반환
    # 7. ORDER BY: 배우 이름 알파벳 순으로 정렬
    actor_filmography_query = """
    // 특정 ID를 가진 배우 노드 찾기
    MATCH (actor:Person)
    WHERE id(actor) IN $actor_ids
    
    // 배우가 출연한 영화 찾기
    MATCH (actor)-[:ACTED_IN]->(movie:Movie)
    
    // 원래 검색된 영화는 제외 (중복 방지)
    WHERE NOT movie.title IN $movie_titles
    
    WITH actor.name as actor_name, collect({
        title: movie.title,
        released: movie.released,
        rating: movie.rating
    }) as other_movies  // 배우별로 출연 영화 정보 수집
    RETURN actor_name, other_movies     // 배우 이름과 출연 영화 목록 반환
    ORDER BY actor_name  // 배우 이름 알파벳 순으로 정렬
    """
    
    actor_filmographies = graph.query(
        actor_filmography_query, 
        params={"actor_ids": actor_ids, "movie_titles": movie_titles}
    )
    
    # 결과 데이터 구조화 - 영화 정보와 배우 필모그래피를 함께 반환
    result = {
        "search_movies": search_movies,
        "actor_filmographies": actor_filmographies
    }
    
    return result

### 2.2 **LCEL** 사용하여 **RAG** 체인을 구성


#### 1) **Graph DB 기반 Retriever 활용**

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# LLM 객체 생성
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.0)

# Prompt 템플릿 정의
template = '''당신은 영화 추천 전문가로서 오직 주어진 정보에 기반하여 객관적이고 정확한 답변을 제공합니다.

[주어진 영화 정보]
{context}

[질문]
{question}

# 답변 작성 지침:
1. 제공된 영화 정보에 명시된 사실만 사용하세요.
2. 간결하고 정확하게 답변하세요.
3. 제공된 정보에 없는 내용은 "제공된 정보에서 해당 내용을 찾을 수 없습니다"라고 답하세요.
4. 영화의 제목, 평점 등 주요 정보를 포함해서 답변하세요.
5. 한국어로 자연스럽고 이해하기 쉽게 답변하세요.
'''

# Prompt 객체 생성
prompt = ChatPromptTemplate.from_template(template)

#### 3) **그래프 경로를 활용한 확장 검색 활용**

In [9]:
# 그래프 경로 기반 검색 함수 정의
def graph_path_search(query, graph_db, k=5):
    """
    벡터 검색으로 영화를 찾고, 그래프 경로를 통해 관련 정보를 확장하는 함수
    
    Args:
        query: 검색 쿼리
        graph_db: 벡터 저장소 객체
        k: 검색할 영화 수
    
    Returns:
        영화와 관련 배우 정보가 포함된 결과
    """
    # 영화와 배우 필모그래피 검색 함수 활용
    search_results = find_movies_and_actors_filmography(
        query=query,
        graph_db=graph_db,
        k=k
    )
    
    # 결과 포맷팅
    formatted_results = []
    
    # 검색된 영화 정보 포맷팅
    formatted_results.append("## 검색된 영화 정보")
    
    # 검색 결과가 있는지 확인
    if not search_results.get("search_movies") or len(search_results["search_movies"]) == 0:
        formatted_results.append("\n검색 결과가 없습니다. 다른 키워드로 검색해보세요.")
    else:
        for movie in search_results["search_movies"]:
            movie_info = f"\n### {movie['movie_title']} ({movie.get('release_date', '정보 없음')})\n"
            movie_info += f"- 평점: {movie.get('rating', '정보 없음')}\n"
            movie_info += f"- 출연 배우: {', '.join(movie.get('actor_names', ['정보 없음']))}\n"
            
            if movie.get('overview'):
                overview = movie['overview']
                movie_info += f"- 줄거리: {overview[:150]}..." if len(overview) > 150 else f"- 줄거리: {overview}\n"
            
            formatted_results.append(movie_info)
    
    # 배우 필모그래피 정보 포맷팅
    if search_results.get("actor_filmographies") and len(search_results["actor_filmographies"]) > 0:
        formatted_results.append("\n## 출연 배우의 다른 작품")
        
        for actor_info in search_results["actor_filmographies"]:
            actor_name = actor_info["actor_name"]
            other_movies = actor_info["other_movies"]
            
            if other_movies and len(other_movies) > 0:
                actor_section = f"\n### {actor_name}의 필모그래피\n"
                
                # 다른 영화 정보 추가 (최대 5개)
                for i, movie in enumerate(other_movies[:5]):
                    actor_section += f"- {movie['title']} ({movie.get('released', '정보 없음')}) - 평점: {movie.get('rating', '정보 없음')}\n"
                
                if len(other_movies) > 5:
                    actor_section += f"- 외 {len(other_movies) - 5}개 작품\n"
                    
                formatted_results.append(actor_section)
            else:
                formatted_results.append(f"\n### {actor_name}의 다른 작품 정보가 없습니다.")
    else:
        formatted_results.append("\n## 배우 필모그래피 정보가 없습니다.")
    
    return "\n".join(formatted_results)

# 그래프 경로 기반 RAG 체인 구성
graph_path_rag_chain = (
    {
        "context": lambda query: graph_path_search(query, graph_vector, k=5),
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [10]:
# 체인 실행
path_query = "2차 세계대전 영화 중에서 가장 평점이 높은 작품과, 그 작품에 출연한 배우들의 다른 작품을 알려주세요"
path_answer = graph_path_rag_chain.invoke(path_query)


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function. ('id' has been replaced by 'elementId or consider using an application-generated id')} {position: line: 19, column: 51, offset: 605} for query: '\n    // 영화 제목 중 하나라도 포함하는 영화 필터링\n    MATCH (movie:Movie)\n    WHERE ANY(title IN $movie_titles WHERE movie.title CONTAINS title)\n\n    // 영화에 출연한 배우가 없을 수도 있으므로 OPTIONAL 사용\n    OPTIONAL MATCH (movie)<-[acted:ACTED_IN]-(actor:Person)\n\n    // 각 영화별로 출연 배우들을 배열로 수집\n    WITH movie, collect(actor) as actors\n\n    // 영화 정보와 배우 정보를 함께 반환\n    RETURN \n        movie.title as movie_title,\n        movie.released as release_date,\n        movie.rating as rating,\n        movie.overview as overview,\n        [actor IN actors | actor.name] as actor_names,   // 배우 이름 배열\n   

벡터스토어 검색된 영화 수: 5


In [11]:
print(path_answer)

2차 세계대전 영화 중에서 가장 평점이 높은 작품은 "The Act of Killing" (평점 7.5)입니다. 다만, 이 영화는 2차 세계대전 배경이 아닌 제노사이드 관련 내용입니다.

2차 세계대전 배경 영화 중 평점이 가장 높은 작품은 "The Good German" (평점 5.9)과 "The Monuments Men" (평점 5.8)입니다. 이 중 평점이 더 높은 "The Good German"에 출연한 배우들의 다른 작품은 다음과 같습니다.

- George Clooney: Syriana(6.3), Ocean's Thirteen(6.5), Michael Clayton(6.5), Leatherheads(5.7), Burn After Reading(6.5) 외 10개 작품  
- Cate Blanchett: Babel(6.9), Notes on a Scandal(6.9), Elizabeth: The Golden Age(6.6), I'm Not There.(6.6), Indiana Jones and the Kingdom of the Crystal Skull(5.7) 외 7개 작품  
- Tobey Maguire: Spider-Man 3(5.9), Brothers(6.8), The Great Gatsby(7.3), Labor Day(6.7)  
- Tony Curran: Underworld: Evolution(6.4)

요약:  
- 2차 세계대전 영화 중 평점 최고: "The Good German" (5.9)  
- 주요 출연 배우 및 다른 작품: George Clooney, Cate Blanchett, Tobey Maguire, Tony Curran (위 목록 참고)  

"The Act of Killing"은 2차 세계대전 영화가 아니므로 제외했습니다.
